In [ ]:
from pynq import Overlay
from pynq import GPIO
import time
import hashlib

In [ ]:
overlay = Overlay("/home/xilinx/cartridge.bit")
c = None

In [ ]:
class Cartridge:
    def __init__(self):
        self.gpio_wr = GPIO(GPIO.get_gpio_pin(1), "out")
        self.gpio_rd = GPIO(GPIO.get_gpio_pin(2), "out")
        self.gpio_cs = GPIO(GPIO.get_gpio_pin(3), "out")
        self.gpio_rst = GPIO(GPIO.get_gpio_pin(4), "out")
        self.gpio_phi = GPIO(GPIO.get_gpio_pin(5), "out")
        self.gpio_vin = GPIO(GPIO.get_gpio_pin(6), "in")
        
        self.gpio_oe = GPIO(GPIO.get_gpio_pin(7), "out")
        self.gpio_dir_A_hi = GPIO(GPIO.get_gpio_pin(8), "out")
        self.gpio_dir_A_lo = GPIO(GPIO.get_gpio_pin(9), "out")
        self.gpio_dir_ctrl = GPIO(GPIO.get_gpio_pin(10), "out")
        self.gpio_dir_D = GPIO(GPIO.get_gpio_pin(11), "out")
        self.gpio_dir_rst = GPIO(GPIO.get_gpio_pin(12), "out")
        self.gpio_dir_vin = GPIO(GPIO.get_gpio_pin(13), "out")

        self.gpio_a = [GPIO(GPIO.get_gpio_pin(14 + i), "out") for i in range(16)]
        self.gpio_d = [GPIO(GPIO.get_gpio_pin(30 + i), "in") for i in range(8)]
        self.read_mode = True
        
        # Set initial directions
        self.gpio_dir_A_hi.write(1) # Output
        self.gpio_dir_A_lo.write(1) # Output
        self.gpio_dir_ctrl.write(1) # Output
        self.gpio_dir_D.write(0) # Input
        self.gpio_dir_rst.write(1) # Output
        self.gpio_dir_vin.write(1) # Input
        self.gpio_oe.write(0) # Enable
        
        time.sleep(0.1)
        
        self.gpio_rst.write(0)
        time.sleep(0.1)
        self.gpio_rst.write(1)
        
    def _write_address(self, address):
        for i in range(16):
            self.gpio_a[i].write(address & 1)
            address = address >> 1
        time.sleep(1.0 / 1_000_000.0)
            
    def read(self, address, rom):
        self.gpio_rd.write(0)
        self.gpio_wr.write(1)
        self.gpio_cs.write(int(rom))
        
        self._write_address(address)
        
        if not self.read_mode:
            self.read_mode = True
            for x in self.gpio_d:
                x.release()
            self.gpio_dir_D.write(0) # Input
            self.gpio_d = [GPIO(GPIO.get_gpio_pin(30 + i), "in") for i in range(8)]
        
        data = 0
        for i in range(8):
            data |= self.gpio_d[i].read() << i
            
        self.gpio_cs.write(1)
            
        return data
            
    def write(self, address, data, rom):
        self.gpio_rd.write(1)
        self.gpio_wr.write(1)
        self.gpio_cs.write(int(rom))
        
        self._write_address(address)
        
        if self.read_mode:
            self.read_mode = False
            for x in self.gpio_d:
                x.release()
            self.gpio_dir_D.write(1) # Output
            self.gpio_d = [GPIO(GPIO.get_gpio_pin(30 + i), "out") for i in range(8)]
            
        for i in range(8):
            self.gpio_d[i].write(data & 1)
            data = data >> 1
            
        self.gpio_wr.write(0)
#         time.sleep(0.5 / 1_000_000.0)
        time.sleep(0.01)
        self.gpio_wr.write(1)
        self.gpio_rd.write(0)
        self.gpio_cs.write(1)
        
    def release(self):
        gpios = [self.gpio_wr, self.gpio_rd, self.gpio_cs, self.gpio_rst] + self.gpio_a + self.gpio_d
        for gpio in gpios:
            gpio.release()
        
if c is not None:
    c.release()
c = Cartridge()

In [ ]:
cartridge_type = c.read(0x0147, True)
print("cart type", hex(cartridge_type))
rom_size = c.read(0x0148, True)
print("rom size", hex(rom_size))
ram_size = c.read(0x0149, True)
print("ram size ", hex(ram_size))

In [ ]:
cart_dump = [c.read(i, True) for i in range(0, 0x4000)]

In [ ]:
# Cartridge type
hex(c.read(0x0147, True))

In [ ]:
hex(c.read(0x4000, True))

In [ ]:
# Rom bank number
c.write(0x2000, 2, True)

In [ ]:
# Enable ram
c.write(0x0000, 0x0A, True)

In [ ]:
hex(c.read(0xA000, False))

In [ ]:
# Disable ram
c.write(0x0000, 0x00, True)

In [ ]:
ram_dumps = []

In [ ]:
ram_dump = []
ram_dumps.append(ram_dump)
start = time.time()
# enable ram
c.write(0x0000, 0x0A, True)
# bank
c.write(0x4000, 0, True)
time.sleep(1)
for i in range(0xA000, 0xA000 + (8 * 1024)):
    b = c.read(i, False)
    ram_dump.append(b)
# disable ram
c.write(0x0000, 0x00, True)
end = time.time()
print("completed in ", end - start)
print("hash: ", hashlib.sha1(bytes(ram_dump)).hexdigest())


print(c.read(0xA000, False))


In [ ]:
with open('/home/xilinx/dump.sav', 'wb') as f:
    f.write(bytes(ram_dump))